# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117676e+02     1.583299e+00
 * time: 0.13747906684875488
     1     1.102450e+01     8.728657e-01
 * time: 0.9790771007537842
     2    -1.102668e+01     1.014251e+00
 * time: 1.1137189865112305
     3    -3.372035e+01     7.771202e-01
 * time: 1.2783639430999756
     4    -4.722426e+01     5.750454e-01
 * time: 1.4314661026000977
     5    -5.661258e+01     2.404362e-01
 * time: 1.5680999755859375
     6    -5.956488e+01     1.580689e-01
 * time: 1.6820719242095947
     7    -6.076515e+01     6.098834e-02
 * time: 1.8100519180297852
     8    -6.126319e+01     6.789682e-02
 * time: 1.9245328903198242
     9    -6.156544e+01     5.681141e-02
 * time: 2.0315558910369873
    10    -6.180139e+01     2.572079e-02
 * time: 2.1455719470977783
    11    -6.197235e+01     2.087375e-02
 * time: 2.2521770000457764
    12    -6.205546e+01     1.736490e-02
 * time: 2.3652050495147705
    13    -6.211607e+01     1.398438e-02
 * time: 2.471721887

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557694
    AtomicNonlocal      14.8522659
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336820

    total               -62.261666459795
